In [196]:
#This cell is for defining various OPTIONS

import os 
#Input the directory where your database is located 
os.chdir('C:/Users/Trond/Documents/Master 2020/Parkinson_PPMI')

import pandas as pd 
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)


In [197]:
#Reading the data table Screening demographics into a pandas dataframe
data = pd.read_csv('_Subject_Characteristics/Screening___Demographics.csv') 
print("Number of total subjects in original data table: ", data.shape[0])

#Discarding indviduals that do not have confirmed parkinson's, Everyone except APPRDX == 1.
data = data.loc[data.APPRDX == 1]
print("Number of subjects with parkinson's in data table:", data.shape[0])

#Selecting only PATNO column and reseting index (0-1-2 instead of 2-6-9). Index was weird due to dropping columns in the previous lines. 
PatNr = data.loc[:, "PATNO"].reset_index(drop = True)

print(PatNr.head())


Number of total subjects in original data table:  2200
Number of subjects with parkinson's in data table: 489
0    3400
1    3403
2    3406
3    3407
4    3150
Name: PATNO, dtype: int64


In [198]:
# Set this flag to 1 if using event_id for longitudinal joining or 0 if using infodt
long_flag = 0
LED = ["INFODT", "EVENT_ID"]


#Importing data tables from non-motor and selecting relevant columns
MOCA = pd.read_csv("Non-motor_Assessments/Montreal_Cognitive_Assessment__MoCA_.csv").loc[:,["PATNO", LED[long_flag], "MCATOT"]]
HPLT = pd.read_csv("Non-motor_Assessments/Hopkins_Verbal_Learning_Test.csv").loc[:,["PATNO", LED[long_flag], "DVT_DELAYED_RECALL"]]
BJLO = pd.read_csv("Non-motor_Assessments/Benton_Judgment_of_Line_Orientation.csv").loc[:,["PATNO", LED[long_flag], "JLO_TOTRAW"]]
LNSQ = pd.read_csv("Non-motor_Assessments/Letter_-_Number_Sequencing__PD_.csv").loc[:,["PATNO", LED[long_flag], "LNS_TOTRAW"]]
SEFL = pd.read_csv("Non-motor_Assessments/Semantic_Fluency.csv").loc[:,["PATNO", LED[long_flag], "VLTANIM", "VLTVEG", "VLTFRUIT"]]
FRBD = pd.read_csv("Non-motor_Assessments/Features_of_REM_Behavior_Disorder.csv").loc[:,["PATNO", LED[long_flag], "RBDDXDT", "RBDDXEST", \
    "ONCLNZP", "ONBENZ", "ONMLATON", "ONSSRI", "ONNORSRI", "ONTRIADP", "ONBTABLK", "REMONEST"]]
GDSS = pd.read_csv("Non-motor_Assessments/Geriatric_Depression_Scale__Short_.csv").loc[:,["PATNO", LED[long_flag], "GDSSATIS", "GDSDROPD", \
    "GDSEMPTY", "GDSBORED", "GDSGSPIR", "GDSAFRAD", "GDSHAPPY", "GDSHLPLS", "GDSHOME", "GDSMEMRY", "GDSALIVE", "GDSWRTLS", "GDSENRGY", \
    "GDSHOPLS", "GDSBETER"]]
SIDT = pd.read_csv("Non-motor_Assessments/University_of_Pennsylvania_Smell_ID_Test.csv").loc[:,["PATNO", "UPSITBK1", "UPSITBK2", \
    "UPSITBK3", "UPSITBK4"]]
EPSS = pd.read_csv("Non-motor_Assessments/Epworth_Sleepiness_Scale.csv").loc[:,["PATNO", LED[long_flag], "ESS1", "ESS2", \
    "ESS3", "ESS4", "ESS5", "ESS6", "ESS7", "ESS8"]]
SCOP = pd.read_csv("Non-motor_Assessments/SCOPA-AUT.csv").loc[:,["PATNO", LED[long_flag], "SCAU1", "SCAU2", \
    "SCAU3", "SCAU4", "SCAU5", "SCAU6", "SCAU7", "SCAU8", "SCAU9", "SCAU10", "SCAU11", "SCAU12", "SCAU13", \
    "SCAU14", "SCAU15", "SCAU16", "SCAU17", "SCAU18", "SCAU19", "SCAU20", "SCAU21", "SCAU22", "SCAU23", "SCAU24", "SCAU25"]]

#Summing up variables from non-motor to a total score and then dropping variables used in summation
SEFL['VLTTOT'] = SEFL.drop(['PATNO', LED[long_flag]], axis = 1).sum(axis = 1)
SEFL = SEFL.loc[:,["PATNO", LED[long_flag], "VLTTOT"]]

FRBD['FRBDTOT'] = FRBD.drop(['PATNO', LED[long_flag]], axis = 1).sum(axis = 1)
FRBD = FRBD.loc[:,["PATNO", LED[long_flag], "FRBDTOT"]]

GDSS['GDSSTOT'] = GDSS.drop(['PATNO', LED[long_flag]], axis = 1).sum(axis = 1)
GDSS = GDSS.loc[:,["PATNO", LED[long_flag], "GDSSTOT"]]

SIDT['SIDTTOT'] = SIDT.drop(['PATNO'], axis = 1).sum(axis = 1)
SIDT = SIDT.loc[:,["PATNO","SIDTTOT"]]

EPSS['EPSSTOT'] = EPSS.drop(['PATNO', LED[long_flag]], axis = 1).sum(axis = 1)
EPSS = EPSS.loc[:,["PATNO", LED[long_flag], "EPSSTOT"]]

#SCOP is not a simple sum and needs additional preprocessing to get total score
# In SCAU1-21, 9 is converted to 3. In SCAU22-25, 9 is converted to 0
for i in range(1,26):
    s = "SCAU" + str(i)
    points = 0
    if i < 22:
        points = 3
    SCOP.loc[SCOP[s] == 9, s] = points
    
SCOP['SCOPTOT'] = SCOP.drop(['PATNO', LED[long_flag]], axis = 1).sum(axis = 1, skipna = True )
SCOP = SCOP.loc[:,["PATNO", LED[long_flag], "SCOPTOT"]]
    



#Importing data tables from motor
MSU3 = pd.read_csv("Motor___MDS-UPDRS/MDS_UPDRS_Part_III.csv").loc[:,["PATNO", LED[long_flag], "NP3SPCH", "NP3FACXP", \
    "NP3RIGN", "NP3RIGRU", "NP3RIGRU", "PN3RIGRL", "NP3RIGLL", "NP3FTAPR", "NP3FTAPL", "NP3HMOVR", "NP3HMOVL", "NP3PRSPR", "NP3PRSPL", \
    "NP3TTAPR", "NP3TTAPL", "NP3TTAPL", "NP3LGAGL", "NP3RISNG", "NP3GAIT", "NP3FRZGT", "NP3PSTBL", "NP3POSTR", "NP3BRADY", "NP3PTRMR", \
    "NP3PTRML", "NP3KTRMR", "NP3KTRML", "NP3RTARU", "NP3RTALU", "NP3RTARL", "NP3RTALL", "NP3RTALJ", "NP3RTCON", "PD_MED_USE"]]
MSU1 = pd.read_csv("Motor___MDS-UPDRS/MDS_UPDRS_Part_I.csv").loc[:,["PATNO", LED[long_flag], "NP1HALL", "NP1COG"]]

#Summing up variables from motor to a total score and then dropping variables used in summation

MSU3['MSU3TOT'] = MSU3.drop(['PATNO', LED[long_flag], "PD_MED_USE"], axis = 1).sum(axis = 1, skipna = True)
MSU3 = MSU3.loc[:,["PATNO", LED[long_flag], "MSU3TOT", "PD_MED_USE"]]


#Importing data tables from subject characteristics
FMHS = pd.read_csv("_Subject_Characteristics/Family_History__PD_.csv").loc[:,["PATNO", "BIOMOMPD", "BIODADPD", \
    "FULSIBPD", "HAFSIBPD", "MAGPARPD", "PAGPARPD", "MATAUPD", "PATAUPD", "KIDSPD"]]
SOEC = pd.read_csv("_Subject_Characteristics/Socio-Economics.csv").loc[:,["PATNO", "EDUCYRS"]]
SCDE = pd.read_csv("_Subject_Characteristics/Screening___Demographics.csv").loc[:,["PATNO", "PRJENRDT", "BIRTHDT"]]



In [199]:
# First step of join: Merge non-longitudinal data
PatNr = pd.DataFrame(PatNr)
data = PatNr.merge(FMHS, how = "inner", on = "PATNO")
data = data.merge(SOEC, how = "inner", on = "PATNO")
data = data.merge(SCDE, how = "inner", on = "PATNO")
data = data.merge(SIDT, how = "inner", on = "PATNO")

#No intersection
#data = data.merge(FRBD, how = "inner", on = "PATNO")


#Second step of join: Merge in longitudinal data

#This data table has duplicate entries for the same data, test is done before drug is administered and after
data = data.merge(MSU3, how = "inner", on ="PATNO")

data = data.merge(MSU1, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(MOCA, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(HPLT, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(BJLO, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(LNSQ, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(SEFL, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(GDSS, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(EPSS, how = "inner", on = ["PATNO", LED[long_flag]])
data = data.merge(SCOP, how = "inner", on = ["PATNO", LED[long_flag]])









In [203]:
import numpy as np
#Inspecting the merged data
np.sort(data.columns.values)
data.shape

(3877, 27)

In [201]:
#Exploring missing values

null_columns=data.columns[data.isnull().any()]
print(data[null_columns].isnull().sum())

print("\nUnique patients entries with BASELINE OR SCREENING")
tempdata = data.loc[(data.EVENT_ID == 'BL') | (data.EVENT_ID == "SC")]
print(tempdata.PATNO.unique().size)

print("\nEntries with missing MCATOT")
print(data[data['MCATOT'].isnull()][["PATNO", LED[long_flag], "MCATOT"]])

print("\nEntries with missing DVT_DELAYED_RECALL")
print(data[data['DVT_DELAYED_RECALL'].isnull()][["PATNO", LED[long_flag], "DVT_DELAYED_RECALL"]])

print("\nEntries with missing JLO_TOTRAW")
print(data[data['JLO_TOTRAW'].isnull()][["PATNO", LED[long_flag], "JLO_TOTRAW"]])




BIOMOMPD                18
BIODADPD                21
FULSIBPD               197
HAFSIBPD              2287
MAGPARPD                24
PAGPARPD                29
MATAUPD                187
PATAUPD                271
KIDSPD                 869
MCATOT                   4
DVT_DELAYED_RECALL       5
JLO_TOTRAW               5
dtype: int64

Unique patients entries with BASELINE OR SCREENING


AttributeError: 'DataFrame' object has no attribute 'EVENT_ID'